Team Jedward Datathon project!

In [4]:
#import values
import pandas as pd

In [8]:
#importing csv files to dataframes
customers_df = pd.read_csv("the_data\olist_customers_dataset.csv")
geolocation_df = pd.read_csv("the_data\olist_geolocation_dataset.csv")
order_items_df = pd.read_csv("the_data\olist_order_items_dataset.csv")
order_payment_df = pd.read_csv("the_data\olist_order_payments_dataset.csv")
order_reviews_df = pd.read_csv("the_data\olist_order_reviews_dataset.csv")
orders_df = pd.read_csv("the_data\olist_orders_dataset.csv")
products_df = pd.read_csv("the_data\olist_products_dataset.csv")
sellers_df = pd.read_csv("the_data\olist_sellers_dataset.csv")